In [ ]:
!wget  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-10-02 16:41:32--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-10-02 16:41:32 (114 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#unzip data
unzip_data('nlp_getting_started.zip')

--2024-10-02 16:41:36--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.130.207, 74.125.68.207, 64.233.170.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.130.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   679KB/s    in 0.9s    

2024-10-02 16:41:37 (679 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
import pandas as pd
train_df=pd.read_csv('/content/train.csv')
test_df=pd.read_csv('/content/test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df_shuffled=train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [ ]:
import random
random_index= random.randint(0, len(train_df)-5)
for row in train_df_shuffled[['text','target']][random_index: random_index+5].itertuples():
  _,text,target=row
  print(f'Target: {target}','(real diaster)' if target>0 else ('not real diaster'))
  print(f'Text:\n {text}')
  print('----\n')


Target: 1 (real diaster)
Text:
 Wreckage 'Conclusively Confirmed' as From MH370: Malaysia PM: Investigators and the families of those who were... http://t.co/pTeVY815mt
----

Target: 0 not real diaster
Text:
 Kevin Tan says hail to the chefs - Û_ and Green Pastures which features sustainable and organic cooking.... http://t.co/D9xVuvp9s6
----

Target: 0 not real diaster
Text:
 @biggangVH1 looks like George was having a panic attack. LOL.
----

Target: 1 (real diaster)
Text:
 Governor weighs parole for California school bus hijacker http://t.co/yFPpIFDkQO http://t.co/aJYUlMFTIF
----

Target: 1 (real diaster)
Text:
 Oops: Bounty hunters try to raid Phoenix police chief's home: http://t.co/yPRJWMigHL -- A group of armed bounty... http://t.co/3RrKRCjYW7
----



In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_sentences, val_sentences, train_labels, val_labels=train_test_split(
    train_df_shuffled['text'].to_numpy(),
    train_df_shuffled['target'].to_numpy(),
    test_size=0.1,
    random_state=42
)

In [ ]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [ ]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
text_vectorizer=TextVectorizer(max_tokens=None,
                               standardize='lower_and_strip_punctuation',
                               split='whitespace',
                               ngrams=None,
                               output_mode='int',
                               output_sequence_length=None,
                               pad_to_max_tokens=True)

ModuleNotFoundError: No module named 'tensorflow.keras.layers.experimental'

In [ ]:
len(train_sentences[0].split())

7

In [ ]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [ ]:
max_vocab_length=10000
max_length=15
text_vectorizer=tf.keras.layers.TextVectorization(max_tokens=max_vocab_length,
                                               output_mode='int',
                                               output_sequence_length=max_length)

In [ ]:
text_vectorizer.adapt(train_sentences)

In [ ]:
sample_sentence= " There's a flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
random_sentence= random.choice(train_sentences)
print(f'original text:\n {random_sentence}\n\n Vectorized version: ')
text_vectorizer([random_sentence])

original text:
 Injuries may be forgiven but not forgotten.

Aesop

 Vectorized version: 


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 267,  133,   21, 3834,   30,   34, 2989,    1,    0,    0,    0,
           0,    0,    0,    0]])>

In [ ]:
words_in_vocab=text_vectorizer.get_vocabulary()
top_5_words=words_in_vocab[:5]
bottom_5_words=words_in_vocab[-5:]
print(f'Most common words in the vocab: {top_5_words}')
print(f'Least common words in the vocab: {bottom_5_words}')

Most common words in the vocab: ['', '[UNK]', 'the', 'a', 'in']
Least common words in the vocab: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [ ]:
len(words_in_vocab)

10000

In [ ]:
from tensorflow.keras import layers
embedding=layers.Embedding(input_dim=max_vocab_length,
                           output_dim=128,
                           embeddings_initializer='uniform',
                           input_length=max_length)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
random_sentences =random.choice(train_sentences)
print(f'Original text:\n {random_sentences}\n\n\n Embedded version: ')
sample_embed = embedding(text_vectorizer([random_sentences]))
sample_embed

Original text:
 Has body bagged ** RT @d_lac: Drake is body bagging meek


 Embedded version: 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03776784,  0.00078364,  0.00251264, ..., -0.04502539,
         -0.02760644,  0.0240631 ],
        [-0.03017653, -0.0444541 , -0.04375688, ...,  0.01558458,
         -0.03299187, -0.00343706],
        [ 0.02528517, -0.00514488,  0.01852927, ...,  0.03688939,
         -0.03980815,  0.04706382],
        ...,
        [ 0.00215999, -0.01899251,  0.01033617, ...,  0.01029531,
          0.02724241, -0.03408588],
        [ 0.00215999, -0.01899251,  0.01033617, ...,  0.01029531,
          0.02724241, -0.03408588],
        [ 0.00215999, -0.01899251,  0.01033617, ...,  0.01029531,
          0.02724241, -0.03408588]]], dtype=float32)>

In [ ]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.03776784,  0.00078364,  0.00251264,  0.00239285, -0.02657678,
       -0.04423738,  0.03991492,  0.04675821, -0.04016184,  0.04909004,
        0.00407854, -0.04737044, -0.02481897,  0.03735681,  0.02220747,
        0.02832792,  0.03272618, -0.00202869, -0.02940246, -0.03193188,
        0.00586389,  0.011071  , -0.03672324, -0.01103268, -0.04643789,
       -0.04182095, -0.04758583, -0.0085101 ,  0.04334596, -0.04905538,
       -0.02279232,  0.01711014,  0.02823569,  0.02113772, -0.02153364,
        0.00857097,  0.03534852, -0.0383988 ,  0.00797384,  0.04656662,
       -0.0441022 , -0.03503355,  0.0436863 , -0.00308565,  0.01265879,
       -0.02699232, -0.01129379,  0.0126779 , -0.02183889, -0.03268182,
        0.01229905,  0.02115734,  0.01017033, -0.01967075,  0.02252647,
       -0.00925042,  0.02035972, -0.03849341,  0.02311325, -0.00679419,
       -0.02033118, -0.00597962, -0.04878889, -0.01559675, -0.03591011,
       -0.029276

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0=Pipeline([(
    'tfidf',TfidfVectorizer()
),
('clf',MultinomialNB())])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
baseline_preds=model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [ ]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_results(y_true,y_pred):
  model_accuracy=accuracy_score(y_true,y_pred)*100
  model_precision, model_recall, model_f1,_= precision_recall_fscore_support(y_true,y_pred, average='weighted')
  model_results={
      'accuracy': model_accuracy,
      'precision':model_precision,
      'recall':model_recall,
      'f1':model_f1
  }
  return model_results

In [ ]:
baseline_results=calculate_results(y_true=val_labels,
                                  y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [ ]:
# create tensorboard callback (need to create a new one for each model)
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import create_tensorboard_callback

SAVE_DIR='model_logs'

--2024-10-02 16:41:52--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-10-02 16:41:52 (78.6 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [ ]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [ ]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,), dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x)
outputs=layers.Dense(1, activation='sigmoid')(x)
model_1=tf.keras.Model(inputs, outputs, name='model_1_dense')

In [ ]:
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

history_1=model_1.fit(train_sentences,
                      train_labels,
                      epochs=5,
                      validation_data=(val_sentences, val_labels),
                      callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                             experiment_name='model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20241002-164914
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.6345 - loss: 0.6493 - val_accuracy: 0.7769 - val_loss: 0.5345
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8182 - loss: 0.4608 - val_accuracy: 0.7822 - val_loss: 0.4806
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8570 - loss: 0.3538 - val_accuracy: 0.7927 - val_loss: 0.4644
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8867 - loss: 0.2913 - val_accuracy: 0.7940 - val_loss: 0.4668
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9106 - loss: 0.2433 - val_accuracy: 0.7848 - val_loss: 0.4771


In [ ]:
model_1.evaluate(val_sentences, val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7728 - loss: 0.5103


[0.47712263464927673, 0.7847769260406494]

In [ ]:
model_1_preds_probs=model_1.predict(val_sentences)
model_1_preds_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


array([[0.40912852],
       [0.81218594],
       [0.99792385],
       [0.09828627],
       [0.12713155],
       [0.93640083],
       [0.9211649 ],
       [0.99483395],
       [0.9645516 ],
       [0.25349784]], dtype=float32)

In [ ]:
model_1_preds=tf.squeeze(tf.round(model_1_preds_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
model_1_results=calculate_results(y_true=val_labels,
                                  y_pred=model_1_preds)
model_1_results

{'accuracy': 78.4776902887139,
 'precision': 0.7883861136419446,
 'recall': 0.7847769028871391,
 'f1': 0.7821779135639718}

In [ ]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [ ]:
import numpy as np
np.array(list(model_1_results.values()))>np.array(list(baseline_results.values()))

array([False, False, False, False])

In [ ]:
words_in_vocab=text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [ ]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization                   │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,840,389 (14.65 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,560,260 (9.77 MB)

In [ ]:
embed_weights=model_1.get_layer('embedding').get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [ ]:
import io
out_v=io.open('vectors.tsv','w', encoding='utf-8')
out_m=io.open('metadata.tsv','w', encoding='utf-8')
for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue
  vec=embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec])+ '\n')
  out_m.write(word + '\n')
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,), dtype='string')
x=text_vectorizer(inputs)
x=embedding(x)
print(f'After embedding: {x.shape}')
# x=layers.LSTM(64, activation='tanh', return_sentences=True)(x)
# print(f'After LSTM cell with return_sentences=True: {x.shape}')
x=layers.LSTM(64, activation='tanh')(x)
print(f'After LSTM cell: {x.shape}')
# x=layers.Dense(1, activation='relu')(x)
outputs=layers.Dense(1, activation='sigmoid')(x)
model_2=tf.keras.Model(inputs, outputs, name='model_2_LSTM')

After embedding: (None, 15, 128)
After LSTM cell: (None, 64)


In [ ]:
model_2.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [ ]:
history_2=model_2.fit(train_sentences,
                      train_labels,
                      epochs=5,
                      validation_data=(val_sentences, val_labels),
                      callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                             experiment_name='model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20241002-173657
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.9023 - loss: 0.3002 - val_accuracy: 0.7874 - val_loss: 0.6121
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9437 - loss: 0.1581 - val_accuracy: 0.7861 - val_loss: 0.5825
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9495 - loss: 0.1296 - val_accuracy: 0.7874 - val_loss: 0.6588
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9559 - loss: 0.1088 - val_accuracy: 0.7835 - val_loss: 0.8219
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9699 - loss: 0.0831 - val_accuracy: 0.7782 - val_loss: 0.8685


In [ ]:
model_2_pred_probs=model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step


((762, 1),
 array([[5.7166195e-03],
        [8.4365231e-01],
        [9.9985433e-01],
        [6.0848881e-02],
        [9.6795976e-04],
        [9.9919122e-01],
        [9.3391985e-01],
        [9.9990427e-01],
        [9.9984813e-01],
        [2.8585717e-01]], dtype=float32))

In [ ]:
model_2_preds=tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
model_2_results=calculate_results(y_true=val_labels,
                                  y_pred=model_2_preds)
model_2_results

{'accuracy': 77.82152230971128,
 'precision': 0.7825342114649019,
 'recall': 0.7782152230971129,
 'f1': 0.7751716074860721}

In [ ]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,), dtype='string')
x=text_vectorizer(inputs)
x=embedding(x)
# x=layers.GRU(64, activation='tanh' return_sentences=True)(x)
#print(x.shape)
x=layers.GRU(64, activation='tanh')(x)
outputs=layers.Dense(1, activation='sigmoid')(x)
model_3=tf.keras.Model(inputs, outputs, name='model_3_GRU')

In [ ]:
model_3.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_3_history=model_3.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences, val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                   experiment_name='model_3_GRU')])

Saving TensorBoard log files to: model_logs/model_3_GRU/20241002-175238
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.8797 - loss: 0.2818 - val_accuracy: 0.7808 - val_loss: 0.6789
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9756 - loss: 0.0726 - val_accuracy: 0.7848 - val_loss: 0.7266
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9733 - loss: 0.0701 - val_accuracy: 0.7743 - val_loss: 0.8660
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9770 - loss: 0.0585 - val_accuracy: 0.7717 - val_loss: 0.9086
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9815 - loss: 0.0475 - val_accuracy: 0.7677 - val_loss: 1.3936


In [ ]:
model_3_pred_probs=model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step


array([[1.6708890e-04],
       [6.8546194e-01],
       [9.9987972e-01],
       [6.8326988e-03],
       [8.6721731e-05],
       [9.9975508e-01],
       [9.3149745e-01],
       [9.9995100e-01],
       [9.9990416e-01],
       [5.4488456e-01]], dtype=float32)

In [ ]:
model_3_preds=tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [ ]:
model_3_results=calculate_results(y_true=val_labels,
                                  y_pred=model_3_preds)
model_3_results

{'accuracy': 76.77165354330708,
 'precision': 0.7746905044137062,
 'recall': 0.7677165354330708,
 'f1': 0.7633360794511403}